# Kaggle Image Recognition Challenge
# Data Downloading Notebook

# Downloading Kaggle Data CSV
The Train data is highly skewed as shown by [this](http://forums.fast.ai/t/how-to-download-data-for-lesson-2-from-kaggle-for-planet-competition/7684) notebook.  



In [1]:
import os
os.getenv("HOME")

'/home/tinkerman'

### Data Downloader Script
CSV File Downloading Script.
Kaggle CLI is installed and used to access competition and download the required csv files.


In [7]:
import os
!pip install kaggle-cli
username = input("Enter Kaggle Username: ")
import getpass
password = getpass.getpass("Enter Kagle Password : ")
!kg config -u {username} -p {password} -c landmark-recognition-challenge
HOME = os.getenv("HOME")
os.makedirs(f'{HOME}/data', exist_ok=True)
os.chdir(f'{HOME}/data')
!kg download
if not os.path.exists("train.csv"):
    !unzip -q sample_submission.csv.zip && unzip -q test.csv.zip && unzip -q train.csv.zip
!ls
os.chdir(HOME)

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Enter Kaggle Username: dumbPy
Enter Kagle Password : ········
downloading https://www.kaggle.com/c/landmark-recognition-challenge/download/test.csv.zip

test.csv.zip already downloaded !
downloading https://www.kaggle.com/c/landmark-recognition-challenge/download/train.csv.zip

train.csv.zip already downloaded !
downloading https://www.kaggle.com/c/landmark-recognition-challenge/download/sample_submission.csv.zip

sample_submission.csv.zip already downloaded !
sample_submission.csv	   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [8]:
!ls data

sample_submission.csv	   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [2]:
import pandas as pd
train = pd.read_csv("data/train.csv")
train.head()

,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231


### Data Downloader Script
Whole Data is downloded to a lower resolution of *299* rather than *1600* to speed up the downoading process, and also because the data was in excess of 350 GB.  
This idea was taken from [this](https://www.kaggle.com/c/landmark-recognition-challenge/discussion/53594) post and the initial download script was taken from [here](https://www.kaggle.com/anokas/python3-dataset-downloader-with-progress-bar) but was changed to take arguments to the function rather than sys.arg\[\]

In [4]:
import os
HOME = os.getenv("HOME")
DATA = f'{HOME}/data/'
output_dirs = [f'{DATA}train', f'{DATA}test']
files = [f'{DATA}train.csv', f'{DATA}test.csv']


import sys, os, multiprocessing, csv
from urllib import request, error
from PIL import Image
from io import BytesIO
import tqdm
from functools import partial
missing_file_index = []
def parse_data(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:2] for line in csvreader]
    return key_url_list[1:]  # Chop off header


def download_image(key_url, out_dir):
    
    (key, url) = key_url
    filename = os.path.join(out_dir, '{}.jpg'.format(key))

    if os.path.exists(filename):
        #print('Image {} already exists. Skipping download.'.format(filename))
        return 0

    try:
        if url[-6:-1] == 's1600': url = url[0:-6] + 's299/'
        response = request.urlopen(url)
        image_data = response.read()
    except:
        #print('Warning: Could not download image {} from {}'.format(key, url))
        missing_file_index.append(key)
        return 1

    try:
        pil_image = Image.open(BytesIO(image_data))
    except:
        #print('Warning: Failed to parse image {}'.format(key))
        return 1

    try:
        pil_image_rgb = pil_image.convert('RGB')
    except:
        #print('Warning: Failed to convert image {} to RGB'.format(key))
        return 1

    try:
        pil_image_rgb.save(filename, format='JPEG')
    except:
        #print('Warning: Failed to save image {}'.format(filename))
        return 1
    
    return 0



def loader(data_file, out_dir):
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    key_url_list = parse_data(data_file)
    pool = multiprocessing.Pool(processes=40)  # Num of CPUs
    failures = sum(tqdm.tqdm(pool.imap_unordered(partial(download_image, out_dir=out_dir), key_url_list), total=len(key_url_list)))
    print('Total number of download failures:', failures)
    pool.close()
    pool.terminate()

    
for i in [0, 1]:
    loader(data_file=files[i], out_dir=output_dirs[i])


100%|██████████| 1217714/1217714 [00:52<00:00, 23310.99it/s]


Total number of download failures: 0


100%|██████████| 117703/117703 [00:15<00:00, 7655.36it/s]

Total number of download failures: 2229


The Above Script was run multiple times to make sure no photo was missed, hence the small run time as it skipped trying for the already downloaded photos.

The actual data download took more than 3 hours.